# Along-slope velocity

Calculate the velocity component tangent to the a bathymetry contour.

Load modules

In [1]:
from pathlib import Path
import intake
import cosima_cookbook as cc
from dask.distributed import Client
import numpy as np
import xarray as xr
import dask

import xgcm
import cf_xarray

# For plotting
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cmocean as cm
import pyproj

By default retain metadata after operations. This can retain out of date metadata, so some caution is required

In [2]:
xr.set_options(keep_attrs=True)

Start a cluster with multiple cores

In [3]:
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 18.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36269,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 18.00 GiB
Comm: tcp://127.0.0.1:34049,Total threads: 1
Dashboard: /proxy/36181/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:44751,


In [4]:
session = cc.database.create_session()

Open the catalogue & define experiment

In [5]:
experiment = '01deg_jra55v13_ryf9091'

In [6]:
catalog = intake.cat.access_nri
cat_filtered = catalog[experiment]
cat_filtered

,unique
path,11947
realm,2
variable,178
frequency,5
start_date,3361
end_date,3361
variable_long_name,181
variable_standard_name,36
variable_cell_methods,3
variable_units,50


Limit to Southern Ocean and single RYF year

In [7]:
latitude_slice  = slice(-80, -59)
start_time = '2086-01-01'
end_time   = '2086-12-31'


Load bathymetry data. Discard the geolon and geolat coordinates: these are 2D curvilinear coordinates that are only required when working above 65N

In [8]:
# hu:  'hu': 'ocean depth on u-cells',
hu_vars = cat_filtered.search(variable='hu')

hu_var_path = hu_vars.df['path'][0]
hu_vars = hu_vars.search(path=hu_var_path)

hu = hu_vars.to_dask()

hu = hu.drop(['geolat_c','geolon_c'])

hu = hu.sel(yu_ocean=latitude_slice).load()['hu']

hu

<xarray.DataArray 'hu' (yu_ocean: 484, xu_ocean: 3600)> Size: 7MB
array([[      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       ...,
       [1685.9166, 1705.9722, 1783.7916, ..., 1646.8873, 1646.8873,
        1685.9166],
       [1552.4305, 1620.2222, 1706.6805, ..., 1541.7136, 1541.7136,
        1552.4305],
       [1459.0416, 1484.2361, 1545.4166, ..., 1494.3055, 1471.2222,
        1459.0416]], dtype=float32)
Coordinates:
  * xu_ocean  (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 4kB -79.99 -79.95 -79.9 ... -59.06 -59.01
Attributes:
    long_name:     ocean depth on u-cells
    units:         m
    valid_range:   [-1.e+09  1.e+09]
    cell_methods:  time: point

Load velocity data, limit to upper 500m and take the mean in time

In [ ]:
uv_data = cat_filtered.search(variable=['u','v'],filename='ocean.nc')
uv_dict = uv_ds.to_dataset_dict().get('ocean.3mon')

In [18]:
uv_data_arr = (
    cat_filtered.search(variable=['u','v'], frequency='3mon')
                .to_dask()
                .sel(yu_ocean=latitude_slice)
                .sel(st_ocean=slice(0,500)).mean('time')
)
u, v = uv_data_arr['u'] , uv_data_arr['v']

u = dask.optimize(u)[0]
u.mean(dim=None).compute()

<xarray.DataArray 'u' ()> Size: 4B
array(0.00858571, dtype=float32)
Attributes:
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_water_x_velocity

In [19]:
u_cc = cc.querying.getvar(experiment, 'u', session, ncfile="ocean.nc",
                       start_time=start_time, end_time=end_time,
                       chunks={}).sel(yu_ocean=latitude_slice).sel(st_ocean=slice(0, 500)).mean('time')
u_cc.mean(dim=None).compute()

<xarray.DataArray 'u' ()> Size: 4B
array(0.00879465, dtype=float32)
Attributes: (12/13)
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_water_x_velocity
    ...             ...
    contact:        Andy Hogg
    email:          andy.hogg@anu.edu.au
    created:        2020-06-11
    description:    0.1 degree ACCESS-OM2 global model configuration with JRA...
    notes:          Additional daily outputs saved from 1 Jan 1950 to 31 Dec ...
    url:            https://github.com/COSIMA/01deg_jra55_ryf/tree/01deg_jra5...

 Currently, migrating from cosima_cookbook => intake results in an increase of the Dask graph layers from 16 => 400 layers, ie. ~25x increase in complexity. This seems like it may well be the root of all the issues surrounding not being able to get the computation to work

 The cells above ([18], [19]) also additionally suggest that we're not looking at exactly the same data when moving from cc => intake.


In [ ]:
u, v = dask.optimize(u)[0], dask.optimize(v)[0] 
u

Load model grid information directly from a grid data file

In [14]:
grid_data_path = Path('/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/')
grid = xr.open_mfdataset(grid_data_path / 'ocean_grid.nc', combine='by_coords').drop(['geolon_t', 'geolat_t', 'geolon_c', 'geolat_c'])

  grid = xr.open_mfdataset(grid_data_path / 'ocean_grid.nc', combine='by_coords').drop(['geolon_t', 'geolat_t', 'geolon_c', 'geolat_c'])



### Along-slope velocity

We calculate the along-slope velocity component by projecting the velocity field to the tangent vector, $u_{along} = \boldsymbol{u \cdot \hat{t}}$, and the cross-slope component by projecting to the normal vector, $v_{cross} = \boldsymbol{u \cdot \hat{n}}$. The schematic below defines the unit normal normal and tangent vectors for a given bathymetric contour, $\boldsymbol{n}$ and $\boldsymbol{t}$ respectively. 

![Sketch of topographic gradient](images/topographic_gradient_sketch.png)

Accordingly, the code below calculates the along-slope velocity component as

$$ u_{along} = (u,v) \boldsymbol{\cdot} \left(\frac{h_y}{|\nabla h|} , -\frac{h_x}{|\nabla h|}\right) = 
u \frac{h_y}{|\nabla h|} - v \frac{h_x}{|\nabla h|}, $$  

and similarly the cross-slope velocity component as

$$ v_{cross} = (u,v) \boldsymbol{\cdot} \left(\frac{h_x}{|\nabla h|} , \frac{h_y}{|\nabla h|}\right)  = 
u \frac{h_x}{|\nabla h|} + v \frac{h_y}{|\nabla h|}.$$ 


We need the derivatives of the bathymetry which we compute using the `xgcm` functionality.

In [15]:
# Give information on the grid: location of u (momentum) and t (tracer) points on B-grid 
ds = xr.merge([hu, grid])
ds.coords['xt_ocean'].attrs.update(axis='X')
ds.coords['xu_ocean'].attrs.update(axis='X', c_grid_axis_shift=0.5)
ds.coords['yt_ocean'].attrs.update(axis='Y')
ds.coords['yu_ocean'].attrs.update(axis='Y', c_grid_axis_shift=0.5)

grid = xgcm.Grid(ds, periodic=['X'])

# Take topographic gradient (simple gradient over one grid cell) and move back to u-grid
dhu_dx = grid.interp( grid.diff(ds.hu, 'X') / grid.interp(ds.dxu, 'X'), 'X')

# In meridional direction, we need to specify what happens at the boundary
dhu_dy = grid.interp( grid.diff(ds.hu, 'Y', boundary='extend') / grid.interp(ds.dyt, 'X'), 'Y', boundary='extend')

# Select latitude slice
dhu_dx = dhu_dx.sel(yu_ocean=latitude_slice)
dhu_dy = dhu_dy.sel(yu_ocean=latitude_slice)

# Magnitude of the topographic slope (to normalise the topographic gradient)
topographic_slope_magnitude = np.sqrt(dhu_dx**2 + dhu_dy**2)

This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg



Calculate along-slope velocity component

In [16]:
# Along-slope velocity
alongslope_velocity = u * dhu_dy / topographic_slope_magnitude - v * dhu_dx / topographic_slope_magnitude
# Attempt to optimize  
alongslope_velocity = dask.optimize(alongslope_velocity)[0]
alongslope_velocity

<xarray.DataArray (st_ocean: 39, yu_ocean: 484, xu_ocean: 3600)> Size: 272MB
dask.array<sub, shape=(39, 484, 3600), dtype=float32, chunksize=(7, 274, 400), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 312B 0.5413 1.681 2.94 ... 383.0 423.7 468.4
  * xu_ocean  (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 4kB -79.99 -79.95 -79.9 ... -59.06 -59.01
Attributes:
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_water_x_velocity

In [ ]:
# Load the data
alongslope_velocity = alongslope_velocity.load()
# warnings might come up in points where we divide by NaN/0,
# i.e., when there is no topographic gradient and warning can be ignored

Vertical averaging (we only need this to plot the velocity on a map)

In [ ]:
# Import edges of st_ocean and add lat/lon dimensions:
st_edges_args = {
    "expt" : experiment,
    "variable" : 'st_edges_ocean',
    "n" : 1,
    "start_time" : start_time,
    "end_time" : end_time,
}
st_edges_ocean = cc.querying.getvar(
    session=session,
    **st_edges_args
)
st_edges_ocean

`st_edges_ocean` isn't in the catalogue - this appears to be because it's a coordinate and 
not a variable.

Not sure if there's a way to access it using intake - for now, we can grab it by directly opening a file 

In [16]:
salt_ds = esm_datastore.search(path='/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/*',variable='salt',frequency='1mon').to_dataset_dict()
salt_ds['ocean.1mon']


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id.frequency'


<xarray.Dataset> Size: 8TB
Dimensions:   (time: 2760, st_ocean: 75, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * xt_ocean  (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * st_ocean  (st_ocean) float64 600B 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) object 22kB 1950-01-16 12:00:00 ... 2179-12-16 12:00:00
Data variables:
    salt      (time, st_ocean, yt_ocean, xt_ocean) float32 8TB dask.array<chunksize=(1, 7, 300, 400), meta=np.ndarray>
Attributes:
    filename:                        ocean.nc
    title:                           ACCESS-OM2-01
    grid_type:                       mosaic
    grid_tile:                       1
    intake_esm_vars:                 ['salt']
    intake_esm_attrs:realm:          ocean
    intake_esm_attrs:frequency:      1mon
    intake_esm_attrs:filename:       ocean.nc
    intake_esm_attrs:file_id:        ocean
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          ocean.1mon

In [17]:
st_edges_data_path = Path('/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/ocean.nc')
st_edges_ds = xr.open_dataset(st_edges_data_path)
st_edges_ocean = st_edges_ds.coords['st_edges_ocean']

In [18]:
st_edges_array = st_edges_ocean.expand_dims({'yu_ocean': u.yu_ocean, 'xu_ocean': u.xu_ocean}, axis=[1, 2])

In [19]:
# Adjust edges at bottom for partial thickness:
st_edges_with_partial = st_edges_array.where(st_edges_array<hu, other=hu)
thickness = st_edges_with_partial.diff(dim='st_edges_ocean')

# Change coordinate of thickness to st_ocean (needed for multipling with other variables):
st_ocean_args = {
    "expt" : experiment,
    "variable" : 'st_ocean',
    "n" : 1,
}
st_ocean = cc.querying.getvar(
    session=session,
    **st_ocean_args
)
thickness['st_edges_ocean'] = st_ocean.values
thickness = thickness.rename(({'st_edges_ocean': 'st_ocean'}))
thickness = thickness.sel(st_ocean=slice(0, 500))

# Depth average gives us the barotropic velocity
barotropic_alongslope_velocity = (alongslope_velocity * thickness).sum('st_ocean') / thickness.sum('st_ocean')

### Plotting

Create a circular path to clip plots

In [20]:
theta  = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.45
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

Create a land mask for plotting, set land cells to 1 and rest to NaN

In [21]:
land = xr.where(np.isnan(hu.rename('land')), 1, np.nan)

#### Map of along-slope velocity with bathymetry contours

In [22]:
fig = plt.figure(1, figsize=(15, 15))

ax = plt.subplot(1, 1, 1, projection=ccrs.SouthPolarStereo(), facecolor="darkgrey")
ax.set_boundary(circle, transform=ax.transAxes)
    
# Filled land 
land.plot.contourf(ax=ax, colors='darkgrey', zorder=2,
                   transform=ccrs.PlateCarree(), add_colorbar=False)

# Coastline
land.fillna(0).plot.contour(ax=ax, colors='k', levels=[0, 1],
                            transform=ccrs.PlateCarree(), add_colorbar=False)

# Depth contours
hu.plot.contour(ax=ax, levels=[500, 1000, 2000, 3000],
                colors='0.2', linewidths=[0.5, 2, 0.5, 0.5], alpha=0.5,
                transform=ccrs.PlateCarree())

# Along slope barotropic velocity
sc = barotropic_alongslope_velocity.plot(ax = ax, cmap=cm.cm.curl,
                                         transform=ccrs.PlateCarree(), vmin=-0.3, vmax=0.3,
                                         cbar_kwargs={'orientation': 'vertical',
                                                      'shrink': 0.25,
                                                      'extend': 'both',
                                                      'label': None,
                                                      'aspect': 8})
  
ax.set_title('Along-slope barotropic velocity (m s$^{-1}$)');

NameError: name 'plt' is not defined

In [ ]:
client.close()